In [ ]:
from midi2audio import FluidSynth
from pydub import AudioSegment


# Inputs
random_seed = 42
n_samples = 10
n_instruments = 3
sampling_rate = 44100


def generate_chord_progression() -> list[str]:
    """Generate chord progression."""


def select_instruments(n_instruments: int) -> list:
    """Select instruments."""


def generate_midi_instrument(chord_progression: list[str], instrument) -> str:
    """Generate MIDI file for each instrument."""


def midi_to_wav(midi_file, wav_file):
    FluidSynth().midi_to_audio(midi_file, wav_file)


def merge_wav_files(inst_wav_files: list[str], merged_wav_file: str):
    """Merge WAV files."""
    mix = None
    for wav_file in inst_wav_files:
        audio = AudioSegment.from_wav(wav_file)
        if mix is None:
            mix = audio
        else:
            mix = mix.overlay(audio)
    mix.export(merged_wav_file, format="wav")


def generate_and_merge_wav_files(random_seed, n_samples, n_instruments, sampling_rate):
    """Generate and merge WAV files."""

In [ ]:
from midi2audio import FluidSynth
from pydub import AudioSegment
import pretty_midi
import random
import os

# Inputs
random_seed = 42
n_samples = 1
n_instruments = 3
sampling_rate = 44100

# Define a directory to store generated MIDI and WAV files
OUTPUT_DIR = "output"
MIDI_DIR = os.path.join(OUTPUT_DIR, "midi")
WAV_DIR = os.path.join(OUTPUT_DIR, "wav")
MERGED_DIR = os.path.join(OUTPUT_DIR, "merged")

os.makedirs(MIDI_DIR, exist_ok=True)
os.makedirs(WAV_DIR, exist_ok=True)
os.makedirs(MERGED_DIR, exist_ok=True)


def generate_chord_progression(length=4) -> list[str]:
    """Generate a random chord progression."""
    chords = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    qualities = ["maj", "min", "dim", "aug"]
    progression = []
    for _ in range(length):
        root = random.choice(chords)
        quality = random.choice(qualities)
        chord = f"{root}{quality}"
        progression.append(chord)
    return progression


def select_instruments(n_instruments: int) -> list[int]:
    """Select n_instruments unique General MIDI program numbers."""
    # General MIDI program numbers range from 0 to 127
    available_instruments = list(range(0, 128))
    selected = random.sample(available_instruments, n_instruments)
    return selected


def chord_to_notes(chord: str) -> list[int]:
    """Convert a chord string to MIDI note numbers."""
    # Simple major and minor chords
    # Note: This is a simplistic implementation
    # In a real scenario, you'd have more comprehensive chord parsing
    chord_map = {"maj": [0, 4, 7], "min": [0, 3, 7], "dim": [0, 3, 6], "aug": [0, 4, 8]}
    # Parse the chord
    for quality in ["maj", "min", "dim", "aug"]:
        if chord.endswith(quality):
            root = chord[: -len(quality)]
            intervals = chord_map[quality]
            break
    else:
        # Default to major
        root = chord
        intervals = chord_map["maj"]
    # Convert root to MIDI number (assuming root octave 4)
    # C4 = 60
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    if root not in note_names:
        root = "C"  # default
    root_index = note_names.index(root)
    root_midi = 60 + root_index  # C4 is 60
    notes = [root_midi + interval for interval in intervals]
    return notes


def generate_midi_instrument(chord_progression: list[str], program: int, filename: str):
    """Generate a MIDI file for a specific instrument and chord progression."""
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    start_time = 0
    chord_duration = 2.0  # seconds per chord

    for chord_str in chord_progression:
        notes = chord_to_notes(chord_str)
        for note_number in notes:
            note = pretty_midi.Note(
                velocity=100,
                pitch=note_number,
                start=start_time,
                end=start_time + chord_duration,
            )
            instrument.notes.append(note)
        start_time += chord_duration

    midi.instruments.append(instrument)
    midi.write(filename)


def midi_to_wav(midi_file, wav_file):
    """Convert MIDI file to WAV using FluidSynth."""
    # Ensure that FluidSynth is initialized with a valid soundfont
    fs = FluidSynth()
    fs.midi_to_audio(midi_file, wav_file)


def merge_wav_files(inst_wav_files: list[str], merged_wav_file: str):
    """Merge WAV files by overlaying them."""
    mix = None
    for wav_file in inst_wav_files:
        audio = AudioSegment.from_wav(wav_file)
        if mix is None:
            mix = audio
        else:
            mix = mix.overlay(audio)
    mix.export(merged_wav_file, format="wav")


def generate_and_merge_wav_files(random_seed, n_samples, n_instruments, sampling_rate):
    """Generate and merge WAV files."""
    random.seed(random_seed)

    # Select instruments once
    instruments = select_instruments(n_instruments)
    print(f"Selected instruments (program numbers): {instruments}")

    for sample_idx in range(n_samples):
        print(f"\nGenerating sample {sample_idx+1}/{n_samples}")
        # Generate chord progression
        chord_progression = generate_chord_progression()
        print(f"Chord progression: {chord_progression}")

        inst_wav_files = []
        for inst_idx, program in enumerate(instruments):
            midi_filename = os.path.join(
                MIDI_DIR, f"sample{sample_idx}_inst{program}.mid"
            )
            wav_filename = os.path.join(
                WAV_DIR, f"sample{sample_idx}_inst{program}.wav"
            )
            # Generate MIDI
            generate_midi_instrument(chord_progression, program, midi_filename)
            print(f"Generated MIDI for instrument {program}: {midi_filename}")
            # Convert MIDI to WAV
            midi_to_wav(midi_filename, wav_filename)
            print(f"Converted to WAV: {wav_filename}")
            inst_wav_files.append(wav_filename)

        # Merge WAV files
        merged_wav_filename = os.path.join(MERGED_DIR, f"sample{sample_idx}_merged.wav")
        merge_wav_files(inst_wav_files, merged_wav_filename)
        print(f"Merged WAV saved to: {merged_wav_filename}")


generate_and_merge_wav_files(random_seed, n_samples, n_instruments, sampling_rate)